0 导入包

In [22]:
# 加载模型
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# 配置
from datasets import load_dataset
import torch
from transformers import SpeechT5HifiGan

# 播放
from IPython.display import Audio as AU

# 计算资源
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

1 加载模型

In [1]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts",device_map={"": 0})

2 配置

In [ ]:
# 需要生成的语音token
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt").to(device)

# 音色嵌入
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# 声码器：频谱图 -> 波形图
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

3 生成并展示

In [6]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
AU(speech, rate=16000)

In [14]:
# 加载模型
from transformers import BarkModel, BarkProcessor

In [19]:
model = BarkModel.from_pretrained("suno/bark-small",device_map={"": 0})
processor = BarkProcessor.from_pretrained("suno/bark-small")

d:\env_config\py\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
d:\env_config\py\lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [25]:
# 需要生成的语音token
inputs = processor("[clears throat] This is a test!", voice_preset="v2/en_speaker_3").to(device) # 后边的参数指定语言和音色
speech_output = model.generate(**inputs).cpu().numpy()
AU(speech_output, rate=16000)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [24]:
inputs = processor( "♪ In the mighty jungle, I'm trying to generate barks.",).to(device)
speech_output = model.generate(**inputs).cpu().numpy()
AU(speech_output, rate=16000)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [26]:
input_list = [
    "[clears throat] Hello uh ..., my dog is cute [laughter]",
    "Let's try generating speech, with Bark, a text-to-speech model",
    "♪ In the jungle, the mighty jungle, the lion barks tonight ♪",
]

# also add a speaker embedding
inputs = processor(input_list, voice_preset="v2/en_speaker_3").to(device)

speech_output = model.generate(**inputs).cpu().numpy()
AU(speech_output[0], rate=16000)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [35]:
from transformers import VitsModel, VitsTokenizer

model = VitsModel.from_pretrained("facebook/mms-tts-deu",device_map={"": 0})
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-deu")

Some weights of the model checkpoint at facebook/mms-tts-deu were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [31]:
text_example = (
    "Ich bin Schnappi das kleine Krokodil, komm aus Ägypten das liegt direkt am Nil."
)
type(text_example)

str

In [33]:
inputs = tokenizer(text_example, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]

In [40]:
with torch.no_grad():
    outputs = model(input_ids)

speech = outputs["waveform"].cpu().numpy()

In [42]:
outputs

VitsModelOutput(waveform=tensor([[nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0'), sequence_lengths=tensor([88832], device='cuda:0'), spectrogram=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0'), hidden_states=None, attentions=None)

In [41]:
AU(speech, rate=16000)